In [97]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

## 1 tool creation: 


In [98]:
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [99]:
print(multiply.invoke({'a':6, 'b':9}))
print(multiply.description)
print(multiply.args)

54
Given 2 numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## 2 tool binding


In [100]:

llm =ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [101]:
llm.invoke('hy')

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--7a508e50-614a-425a-9ca6-93fe37dce0b6-0', usage_metadata={'input_tokens': 2, 'output_tokens': 9, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}})

In [102]:
llm_with_tools=llm.bind_tools([multiply])

## 3 tool calling

In [103]:
llm_with_tools.invoke('hy')

# here tool is not call because not need tool

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--f98056d0-1873-4fec-98db-e6347a8889ff-0', usage_metadata={'input_tokens': 55, 'output_tokens': 9, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})

In [104]:
message=[]

In [105]:
query=HumanMessage('can you multiply 8 with 1000')

In [106]:
message.append(query)

In [107]:
message

[HumanMessage(content='can you multiply 8 with 1000', additional_kwargs={}, response_metadata={})]

In [108]:
llm.invoke(message)

# here llm don't have tool

AIMessage(content='To multiply 8 by 1000, you just add three zeros to the 8:\n\n8 * 1000 = **8000**', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--6c667021-e614-46df-b865-8492cf32f135-0', usage_metadata={'input_tokens': 12, 'output_tokens': 36, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}})

In [109]:
result=llm_with_tools.invoke(message)

# here tool call happen 

In [110]:
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 8.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--78f33573-4d9e-4783-b7f5-321f75e91b97-0', tool_calls=[{'name': 'multiply', 'args': {'a': 8.0, 'b': 1000.0}, 'id': 'bb6ba8f4-15a8-4e4d-8cea-9f4c77be940a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 21, 'total_tokens': 165, 'input_token_details': {'cache_read': 0}})

In [111]:
# content='' additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 8.0, "b": 1000.0}'}} 
# response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} 
# id='run--6ed68018-3148-4395-9ec0-71eb86b8077a-0' tool_calls=[{'name': 'multiply', 'args': {'a': 8.0, 'b': 1000.0},                                                           'id': 'a20d0eeb-bd4e-4fa7-bd3c-6f8686cd9b31', 'type': 'tool_call'}] 
# usage_metadata={'input_tokens': 65, 'output_tokens': 21, 'total_tokens': 235, 'input_token_details': {'cache_read': 0}}


In [112]:
message.append(result)

In [113]:
message

[HumanMessage(content='can you multiply 8 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 8.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--78f33573-4d9e-4783-b7f5-321f75e91b97-0', tool_calls=[{'name': 'multiply', 'args': {'a': 8.0, 'b': 1000.0}, 'id': 'bb6ba8f4-15a8-4e4d-8cea-9f4c77be940a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 21, 'total_tokens': 165, 'input_token_details': {'cache_read': 0}})]

In [114]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 8.0, 'b': 1000.0},
  'id': 'bb6ba8f4-15a8-4e4d-8cea-9f4c77be940a',
  'type': 'tool_call'}]

In [115]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 8.0, 'b': 1000.0},
 'id': 'bb6ba8f4-15a8-4e4d-8cea-9f4c77be940a',
 'type': 'tool_call'}

In [116]:
result.tool_calls[0]['args']

{'a': 8.0, 'b': 1000.0}

## # 4 tool execution: 

In [117]:

multiply.invoke(result.tool_calls[0]['args'])
# we don't do like this 

8000

In [118]:
tool_result=multiply.invoke(result.tool_calls[0])

# we send this to the llm : 

# {'name': 'multiply',
#  'args': {'a': 8.0, 'b': 45.0},
#  'id': '59b18687-eee9-424e-ac3d-9ab58221808d',
#  'type': 'tool_call'}


In [119]:
tool_result

ToolMessage(content='8000', name='multiply', tool_call_id='bb6ba8f4-15a8-4e4d-8cea-9f4c77be940a')

In [120]:
message.append(tool_result)

In [121]:
message

[HumanMessage(content='can you multiply 8 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 8.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--78f33573-4d9e-4783-b7f5-321f75e91b97-0', tool_calls=[{'name': 'multiply', 'args': {'a': 8.0, 'b': 1000.0}, 'id': 'bb6ba8f4-15a8-4e4d-8cea-9f4c77be940a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 21, 'total_tokens': 165, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='8000', name='multiply', tool_call_id='bb6ba8f4-15a8-4e4d-8cea-9f4c77be940a')]

In [ ]:
llm_with_tools.invoke(message).content

# to see the result in the good format

'8 multiplied by 1000 is 8000.'